In [165]:
%matplotlib inline
import pykat
import pylab as pl
import pandas
import matplotlib.pyplot as plt
import matplotlib
import csv
import math
import numpy as np
from pykat.parallel import parakat

pandas.set_eng_float_format(accuracy=3, use_eng_prefix=True)

In [166]:
#extract the amplitude and beam profile from the input at the BS.

katfile = "aLIGO_IFO_AWC_MM.kat"
basekat = pykat.finesse.kat()
basekat.loadKatFile(katfile)
basekat.parseCommands("""
ad inputa 0 nPRBS

bp inputxr x r nPRBS
bp inputyr y r nPRBS

bp inputxw x w nPRBS
bp inputyw y w nPRBS

""")

basekat.undumpNodes() #undump the vacuum nodes
kat=basekat.deepcopy() #copy kat code
kat.noxaxis = True
kat.verbose = False

out = kat.run(getTraceData=True)

Parsing `put lx1 L $Lmx` into pykat object not implemented yet, added as extra line.
Parsing `put ly1 L $Lmy` into pykat object not implemented yet, added as extra line.


In [167]:
#### Extract the beam radius and curvature incident on BS
inputxw = float(np.real(out[0]['inputxw']))
inputxr = float(np.real(out[0]['inputxr']))

inputyw = float(np.real(out[0]['inputyw']))
inputyr = float(np.real(out[0]['inputyr']))

### Calculate the input power to match real aLIGO IFO
c = 3.0e8
n = 1.0
eps = 8.854e-12

input_I = (float(np.absolute(out[0]['inputa'])**2)) * (c * n * eps/2 )
           
avg_w = (inputxw +inputyw)/2

input_P = input_I * 4 *np.pi * avg_w**2

In [212]:
katfile1 = "aLIGO_IFO_AWC_MM_slim.kat"

basekat1 = pykat.finesse.kat()
basekat1.loadKatFile(katfile1)

# Add cavity parameter detectors
basekat1.parseCommands("""
bp itmxq x q nITMX2

bp wx x w nYBS
bp wy y w nYBS
bp rcx x r nYBS
bp rcy y r nYBS

#cp cavOMC x w
#cp cavOMC x w0
#cp cavOMC x z
#cp cavOMC x r
#cp cavOMC x q
#cp cavOMC x length
#cp cavOMC x loss

yaxis lin re:im
""")

"""cp cavXARM x w
cp cavXARM x w0
cp cavXARM x z
cp cavXARM x r
cp cavXARM x q
cp cavXARM x length
cp cavXARM x loss

cp cavYARM x w
cp cavYARM x w0
cp cavYARM x z
cp cavYARM x r
cp cavYARM x q
cp cavYARM x length
cp cavYARM x loss"""


#define the gaussian beam parameters incident on BS from PRM
gauss_string = 'gauss gaussin L0 nPRBS ' + str(inputxw) + ' ' \
+ str(inputxr)+' '\
+ str(inputyw) + ' '\
+ str(inputyr)
basekat1.parseCommands(gauss_string)

basekat1.undumpNodes() #undump the vacuum nodes
kat1=basekat1.deepcopy() #copy kat code
kat1.noxaxis = True
kat1.verbose = False

#Turn off everthing except for OMC, Finesse will work very hard to get the right parameters
#to lock the OMC.
for cav in kat1.getAll(pykat.commands.cavity):
    cav.enabled=False
          
#kat1.cavOMC.enabled=False
kat1.cavXARM.enabled=True
kat1.cavYARM.enabled=True


kat1.L0._laser__power._value = input_P

kat1.SRMHR.Rc = -5.6938
out1  = kat1.run(getTraceData=True)

kat1.SRMHR.Rc = 'inf'
out2 = kat1.run(getTraceData=True)

kat1.SRMHR.Rc.value = -6.0
out3 = kat1.run(getTraceData=True)

Parsing `put lx1 L $Lmx` into pykat object not implemented yet, added as extra line.
Parsing `put ly1 L $Lmy` into pykat object not implemented yet, added as extra line.


In [192]:
omc_z_1 = np.real(out1[0]['cavOMC_x_z'])
print(omc_z_1)

omc_q_1 = out1[0]['cavOMC_x_q']
print(omc_q_1)

omc_r_1 = np.real(out1[0]['cavOMC_x_r'])
print(omc_r_1)

omc_w_1 = np.real(out1[0]['cavOMC_x_w'])
print(omc_w_1)

omc_l_1 = np.real(out1[0]['cavOMC_x_length'])
print(omc_l_1)

omc_l_1 = np.real(out1[0]['cavOMC_x_loss'])
print(omc_l_1)

0.140764742971947
(0.140764742972+0.707540463523j)
3.69714894084054
0.000499115038950642
1.1314
0.0151533625445898


In [188]:
# doublecheck q values
lam = 1064e-9
pi = 3.1415926
q_check =1/( (1/omc_r_1) - 1j * (lam/(pi*omc_w_1**2)) )
print(q_check)

(0.140764738352+0.707540452373j)


In [215]:
#look at nodal trace of full IFO
print(out1[1][0]['nSRMHRa'][0].q)
print(out2[1][0]['nSRMHRa'][0].q)
print(out3[1][0]['nSRMHRa'][0].q)

(3.35258124181355+0.913594763941617j)
(3.35258124181355+0.913594763941617j)
(3.35258124181355+0.913594763941617j)


In [214]:
print(out1[1][0]['nSRMHRa'][0].q)

(3.35258124181355+0.913594763941617j)
